In [3]:
# Imports
import requests
from bs4 import BeautifulSoup

import pandas as pd

from gensim.summarization import summarize
from gensim.summarization import keywords

In [4]:
#read data file
df = pd.read_csv("NytApiData.csv")

In [5]:
#drop problem rows
df = df.drop([18,33])
df.reset_index(inplace = True, drop = True)

In [6]:
#pulls page based on URL
def GetPage(Url):
    page = requests.get(Url).text
    soup = BeautifulSoup(page)
    return soup

#function for web scraper 
def GetText(soup):
    # Get text from all <p> tags.
    p_tags = soup.find_all('p')
    # Get the text from each of the “p” tags and strip surrounding whitespace.
    p_tags_text = [tag.get_text().strip() for tag in p_tags]
    # Get text from all <p> tags.
    p_tags = soup.find_all('p')
    # Get the text from each of the “p” tags and strip surrounding whitespace.
    p_tags_text = [tag.get_text().strip() for tag in p_tags]
    # Filter out sentences that contain newline characters '\n' or don't contain periods.
    sentence_list = [sentence for sentence in p_tags_text if not '\n' in sentence]
    sentence_list = [sentence for sentence in sentence_list if '.' in sentence]
    # Combine list items into string.
    article = ' '.join(sentence_list)
    return article

#funtion to summarize scraped data
Sum = lambda x : summarize(x, ratio=0.3)
    

In [7]:
#Calls, formats, and returns scraped data into a dictonary
def GetArticalSummarys(df):
    ret = []
    i = 0
    for index, row in df.iterrows():
        temp = {}
        temp['Date'] = df.loc[index,"Date"]
        url = df.loc[index,"URL"]
        soup = GetPage(url)
        article = GetText(soup)
        if len(article) > 0:
            temp['KeyWords'] = keywords(article)
            temp['Artical'] = article
            temp['good'] = 'yes'
        else:
            temp['KeyWords'] = "None"
            temp['Artical'] = "None"
        
        temp['Source'] = df.loc[index,"Source"]
        temp['News Desk'] = df.loc[index,"News Desk"]
        temp['Headline'] = df.loc[index,"Headline"]
        temp['Abstract'] = df.loc[index,"Abstract"]
        temp['Subjects'] = df.loc[index,"Subjects"]
        ret.append(temp)
    return ret
    

In [8]:
out = GetArticalSummarys(df)

In [10]:
#turn dict intp dataframe
outdf = pd.DataFrame.from_dict(out)

In [12]:
#drop bad rows
outdf.drop(outdf.index[outdf['good'] != 'yes'], inplace = True)

In [56]:
#save data
outdf.to_csv('ArticalData.csv')